# Models:
1. RNN with Attention Layer (Fibonacci Sequence)
2. RNN with Attention Layer (Lucas Sequence)

In [1]:
import numpy as np
from keras import Model
from keras.layers import Layer
from keras.layers import Input, Dense, SimpleRNN
import tensorflow as tf                                # Tensorflow META Package: tf
import tensorflow.keras.activations as tfa             # Tensorflow -> Keras -> Activations = tfa
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.metrics import mean_squared_error

In [2]:
# Prepare the dataset
# Generate the Fibonacci Sequence
def get_fib_seq(n, scale_data=True):
    # Get the Fibonacci sequence
    seq = np.zeros(n)
    fib_n1 = 0.0
    fib_n = 1.0
    for i in range(n):
        seq[i] = fib_n1 + fib_n
        fib_n1 = fib_n
        fib_n = seq[i]
    scaler = []
    if scale_data:
        scaler = MinMaxScaler(feature_range=(0, 1))
        seq = np.reshape(seq, (n, 1))
        seq = scaler.fit_transform(seq).flatten()
    return seq, scaler

fib_seq = get_fib_seq(10, False)[0]
print(fib_seq)

[ 1.  2.  3.  5.  8. 13. 21. 34. 55. 89.]


In [3]:
# Get Compile Training Examples and Target values
def get_fib_XY(total_fib_numbers, time_steps, train_percent, scale_data=True):
    dat, scaler = get_fib_seq(total_fib_numbers, scale_data)
    Y_ind = np.arange(time_steps, len(dat), 1)
    Y = dat[Y_ind]
    rows_x = len(Y)
    X = dat[0:rows_x]
    for i in range(time_steps-1):
        temp = dat[i+1:rows_x+i+1]
        X = np.column_stack((X, temp))
    # random permutation with fixed seed
    rand = np.random.RandomState(seed=13)
    idx = rand.permutation(rows_x)
    split = int(train_percent*rows_x)
    train_ind = idx[0:split]
    test_ind = idx[split:]
    trainX = X[train_ind]
    trainY = Y[train_ind]
    testX = X[test_ind]
    testY = Y[test_ind]
    trainX = np.reshape(trainX, (len(trainX), time_steps, 1))
    testX = np.reshape(testX, (len(testX), time_steps, 1))
    return trainX, trainY, testX, testY, scaler

trainX1, trainY1, testX1, testY1, scaler = get_fib_XY(12, 3, 0.7, False)
print('trainX = ', trainX1)
print('trainY = ', trainY1)

trainX =  [[[ 8.]
  [13.]
  [21.]]

 [[ 5.]
  [ 8.]
  [13.]]

 [[ 2.]
  [ 3.]
  [ 5.]]

 [[13.]
  [21.]
  [34.]]

 [[21.]
  [34.]
  [55.]]

 [[34.]
  [55.]
  [89.]]]
trainY =  [ 34.  21.   8.  55.  89. 144.]


In [4]:
# Function to create Simple RNN
def create_RNN(hidden_units, dense_units, input_shape, activation):
    model = Sequential()
    model.add(SimpleRNN(hidden_units, input_shape=input_shape, activation=activation[0]))
    model.add(Dense(units=dense_units, activation=activation[1]))
    model.compile(loss='mse', optimizer='adam')
    return model

# 1. Predefined parameters
time_steps = 20
hidden_units = 2
epochs = 30

# 2. Generate dataset
trainX1, trainY1, testX1, testY1, scaler = get_fib_XY(1200, time_steps, 0.7)

# 3. Model Created: model_RNN
model_RNN = create_RNN(hidden_units=hidden_units, dense_units=1, input_shape=(time_steps,1),
activation=['tanh', 'tanh'])

# 3.1 Visualize
# model_RNN.summary()

# 4. Train the network
model_RNN.fit(trainX1, trainY1, epochs=epochs, batch_size=1, verbose=2)

Epoch 1/30


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


826/826 - 4s - 5ms/step - loss: 0.0038
Epoch 2/30
826/826 - 4s - 4ms/step - loss: 0.0035
Epoch 3/30
826/826 - 4s - 4ms/step - loss: 0.0034
Epoch 4/30
826/826 - 3s - 3ms/step - loss: 0.0032
Epoch 5/30
826/826 - 3s - 3ms/step - loss: 0.0031
Epoch 6/30
826/826 - 5s - 6ms/step - loss: 0.0029
Epoch 7/30
826/826 - 8s - 9ms/step - loss: 0.0028
Epoch 8/30
826/826 - 4s - 4ms/step - loss: 0.0026
Epoch 9/30
826/826 - 3s - 3ms/step - loss: 0.0026
Epoch 10/30
826/826 - 5s - 6ms/step - loss: 0.0025
Epoch 11/30
826/826 - 3s - 4ms/step - loss: 0.0023
Epoch 12/30
826/826 - 4s - 5ms/step - loss: 0.0022
Epoch 13/30
826/826 - 3s - 3ms/step - loss: 0.0021
Epoch 14/30
826/826 - 6s - 7ms/step - loss: 0.0021
Epoch 15/30
826/826 - 4s - 5ms/step - loss: 0.0020
Epoch 16/30
826/826 - 5s - 6ms/step - loss: 0.0019
Epoch 17/30
826/826 - 5s - 6ms/step - loss: 0.0018
Epoch 18/30
826/826 - 3s - 3ms/step - loss: 0.0018
Epoch 19/30
826/826 - 6s - 7ms/step - loss: 0.0017
Epoch 20/30
826/826 - 3s - 4ms/step - loss: 0.0017


In [5]:
# Evaluate model

# 5. Evaluate model
train_mse = model_RNN.evaluate(trainX1, trainY1)
test_mse = model_RNN.evaluate(testX1, testY1)

# 6. Print error
print("Train set MSE = ", train_mse)
print("Test set MSE = ", test_mse)

26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0014
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 9.1784e-04
Train set MSE =  0.0014474530471488833
Test set MSE =  0.0012856258545070887


In [6]:
# Function to Build the Attention layer
class attention(Layer):
    def __init__(self,**kwargs):
        super(attention,self).__init__(**kwargs)

    def build(self,input_shape):
        self.W=self.add_weight(name='attention_weight', shape=(input_shape[-1],1),
        initializer='random_normal', trainable=True)
        self.b=self.add_weight(name='attention_bias', shape=(input_shape[1],1),
        initializer='zeros', trainable=True)
        super(attention, self).build(input_shape)

    def call(self,x):
        # Alignment scores. Pass them through tanh function
        e = tfa.tanh(tf.matmul(x,self.W)+self.b)
        # Remove dimension of size 1
        e = tf.squeeze(e, axis=-1)
        # Compute the weights
        alpha = tfa.softmax(e)
        # Reshape to tensorFlow format
        alpha = tf.expand_dims(alpha, axis=-1)
        # Compute the context vector
        context = x * alpha
        context = tf.reduce_sum(context, axis=1)
        return context

# Function to Create RNN with the attention layer
def create_RNN_with_attention(hidden_units, dense_units, input_shape, activation):
    x=Input(shape=input_shape)
    RNN_layer = SimpleRNN(hidden_units, return_sequences=True, activation=activation)(x)
    attention_layer = attention()(RNN_layer)
    outputs=Dense(dense_units, trainable=True, activation=activation)(attention_layer)
    model=Model(x,outputs)
    model.compile(loss='mse', optimizer='adam')
    return model

In [7]:
# 1. Dataset has already been generated

# 2. Model Created: RNN with Attention Layer
model1 = create_RNN_with_attention(hidden_units=hidden_units, dense_units=1,input_shape=(time_steps,1), activation='tanh')

# 2.1 Visualize:
# model_attention.summary()

# 3. Train
model1.fit(trainX1, trainY1, epochs=epochs, batch_size=1, verbose=2)

# 4. Evaluate
train_mse1 = model1.evaluate(trainX1, trainY1)
test_mse1 = model1.evaluate(testX1, testY1)

# 5. Print error
print("Train set MSE with attention = ", train_mse1)
print("Test set MSE with attention = ", test_mse1)

Epoch 1/30
826/826 - 6s - 7ms/step - loss: 0.0014
Epoch 2/30
826/826 - 3s - 4ms/step - loss: 0.0014
Epoch 3/30
826/826 - 5s - 6ms/step - loss: 0.0014
Epoch 4/30
826/826 - 3s - 3ms/step - loss: 0.0014
Epoch 5/30
826/826 - 3s - 4ms/step - loss: 0.0013
Epoch 6/30
826/826 - 5s - 7ms/step - loss: 0.0013
Epoch 7/30
826/826 - 3s - 4ms/step - loss: 0.0013
Epoch 8/30
826/826 - 3s - 4ms/step - loss: 0.0013
Epoch 9/30
826/826 - 4s - 5ms/step - loss: 0.0012
Epoch 10/30
826/826 - 4s - 5ms/step - loss: 0.0012
Epoch 11/30
826/826 - 3s - 3ms/step - loss: 0.0011
Epoch 12/30
826/826 - 6s - 7ms/step - loss: 0.0011
Epoch 13/30
826/826 - 3s - 4ms/step - loss: 0.0011
Epoch 14/30
826/826 - 6s - 7ms/step - loss: 0.0010
Epoch 15/30
826/826 - 5s - 6ms/step - loss: 9.5445e-04
Epoch 16/30
826/826 - 3s - 3ms/step - loss: 9.0647e-04
Epoch 17/30
826/826 - 3s - 3ms/step - loss: 8.4923e-04
Epoch 18/30
826/826 - 3s - 4ms/step - loss: 7.9672e-04
Epoch 19/30
826/826 - 3s - 4ms/step - loss: 7.4330e-04
Epoch 20/30
826/826 

# Task: Build a custom layer in RNN to predict the next Lucas number.
- Create a function to generate the Lucas sequence.
- Create a class to implement the Lucas layer.
- Integrate the custom layer in simple RNN.
- Show the performance of the model.

In [8]:
# Function to generate Lucas layers
class LucasLayer(Layer):
    def __init__(self, units, **kwargs):
        super(LucasLayer, self).__init__(**kwargs)
        self.units = units

    def build(self, input_shape):
        self.kernel = self.add_weight(shape=(input_shape[-1], self.units),
                                      initializer='uniform',
                                      name='kernel')
        self.bias = self.add_weight(shape=(self.units,),
                                    initializer='zeros',
                                    name='bias')
        super(LucasLayer, self).build(input_shape)

    def call(self, inputs):
        lucas_numbers = [2, 1]
        for _ in range(self.units - 2):
            next_lucas = lucas_numbers[-1] + lucas_numbers[-2]
            lucas_numbers.append(next_lucas)

        output = tf.matmul(inputs, self.kernel) + self.bias
        return output

# Function to create the RNN with the additional lucas layer
def create_RNN_with_lucas(hidden_units, dense_units, input_shape, activation):
    model = Sequential()
    model.add(SimpleRNN(hidden_units, input_shape=input_shape, activation=activation[0]))
    model.add(LucasLayer(units=hidden_units))
    model.add(Dense(units=dense_units, activation=activation[1]))
    model.compile(loss='mse', optimizer='adam')
    return model


def get_sequence_data(seq_type, total_numbers, time_steps, train_percent, scale_data=True):
    def generate_sequence(n):
        seq = np.zeros(n)
        if seq_type == 'fib':
            n1, n2 = 0.0, 1.0
        elif seq_type == 'lucas':
            n1, n2 = 2.0, 1.0
        else:
            raise ValueError("Invalid seq_type. Choose 'fib' or 'lucas'.")

        for i in range(n):
            seq[i] = n1 + n2
            n1, n2 = n2, seq[i]
        return seq

    dat = generate_sequence(total_numbers)
    scaler = []
    if scale_data:
        scaler = MinMaxScaler(feature_range=(0, 1))
        dat = np.reshape(dat, (total_numbers, 1))
        dat = scaler.fit_transform(dat).flatten()

    Y_ind = np.arange(time_steps, len(dat), 1)
    Y = dat[Y_ind]
    rows_x = len(Y)
    X = dat[0:rows_x]
    for i in range(time_steps - 1):
        temp = dat[i + 1:rows_x + i + 1]
        X = np.column_stack((X, temp))

    rand = np.random.RandomState(seed=13)
    idx = rand.permutation(rows_x)
    split = int(train_percent * rows_x)
    train_ind = idx[0:split]
    test_ind = idx[split:]
    trainX = X[train_ind]
    trainY = Y[train_ind]
    testX = X[test_ind]
    testY = Y[test_ind]
    trainX = np.reshape(trainX, (len(trainX), time_steps, 1))
    testX = np.reshape(testX, (len(testX), time_steps, 1))
    return trainX, trainY, testX, testY, scaler

In [9]:
# 1. Predefined parameters
time_steps = 20
hidden_units = 2
epochs = 20

# 2. Generate Dataset
trainX2, trainY2, testX2, testY2, scaler_lucas = get_sequence_data('lucas', 1200, time_steps, 0.7)

# 3. Model Created: model_RNN_with_lucas
model2 = create_RNN_with_lucas(hidden_units=hidden_units, dense_units=1, input_shape=(time_steps,1), activation=['tanh', 'tanh'])

# 3.1 Visualize
# model2.summary()

# 4. Train the network
model2.fit(trainX2, trainY2, epochs=epochs, batch_size=1, verbose=2)

Epoch 1/20


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


826/826 - 6s - 7ms/step - loss: 0.0014
Epoch 2/20
826/826 - 4s - 5ms/step - loss: 0.0014
Epoch 3/20
826/826 - 6s - 7ms/step - loss: 0.0014
Epoch 4/20
826/826 - 3s - 4ms/step - loss: 0.0014
Epoch 5/20
826/826 - 3s - 3ms/step - loss: 0.0014
Epoch 6/20
826/826 - 3s - 3ms/step - loss: 0.0013
Epoch 7/20
826/826 - 6s - 8ms/step - loss: 0.0013
Epoch 8/20
826/826 - 3s - 3ms/step - loss: 0.0012
Epoch 9/20
826/826 - 5s - 6ms/step - loss: 0.0011
Epoch 10/20
826/826 - 4s - 5ms/step - loss: 9.3623e-04
Epoch 11/20
826/826 - 4s - 5ms/step - loss: 8.0492e-04
Epoch 12/20
826/826 - 5s - 6ms/step - loss: 6.6038e-04
Epoch 13/20
826/826 - 5s - 6ms/step - loss: 5.1216e-04
Epoch 14/20
826/826 - 3s - 4ms/step - loss: 3.9681e-04
Epoch 15/20
826/826 - 6s - 7ms/step - loss: 2.7191e-04
Epoch 16/20
826/826 - 3s - 3ms/step - loss: 1.8686e-04
Epoch 17/20
826/826 - 5s - 6ms/step - loss: 1.4038e-04
Epoch 18/20
826/826 - 3s - 4ms/step - loss: 1.1477e-04
Epoch 19/20
826/826 - 5s - 6ms/step - loss: 1.0848e-04
Epoch 20/20

In [10]:
# Evaluate model

# 5. Evaluate model
train_mse2 = model2.evaluate(trainX2, trainY2)
test_mse2 = model2.evaluate(testX2, testY2)

# 6. Print error
print("Train set MSE with Lucas Layer= ", train_mse2)
print("Test set MSE with Lucas Layer= ", test_mse2)

26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 5.7928e-05
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 2.0502e-05 
Train set MSE with Lucas Layer=  6.103795749368146e-05
Test set MSE with Lucas Layer=  8.992853508971166e-06


In [11]:
# Export to PDF
# !apt install -y texlive-xetex texlive-fonts-recommended texlive-latex-extra pandoc
!jupyter nbconvert --to pdf "10.ipynb"

from google.colab import files
files.download("10.pdf")

[NbConvertApp] WARNING | pattern '9.ipynb' matched no files
This application is used to convert notebook files (*.ipynb)
        to various other formats.


Options
The options below are convenience aliases to configurable class-options,
as listed in the "Equivalent to" description-line of the aliases.
To see all configurable class-options for some <cmd>, use:
    <cmd> --help-all

--debug
    set log level to logging.DEBUG (maximize logging output)
    Equivalent to: [--Application.log_level=10]
--show-config
    Show the application's configuration (human-readable format)
    Equivalent to: [--Application.show_config=True]
--show-config-json
    Show the application's configuration (json format)
    Equivalent to: [--Application.show_config_json=True]
--generate-config
    generate default config file
    Equivalent to: [--JupyterApp.generate_config=True]
-y
    Answer yes to any questions instead of prompting.
    Equivalent to: [--JupyterApp.answer_yes=True]
--execute
    Execute t

FileNotFoundError: Cannot find file: 9.pdf